In [1]:
import pandas as pd
import random
import re
import json

random.seed(2)

which_wiki = 4

# Read in the data
df = pd.read_csv(f'all-outputs-new-wiki-{which_wiki}.csv')

In [2]:
import random

def pairwise(annotators):
    pairs = []
    n = len(annotators)
    for i in range(n):
        for j in range(i+1, n):
            pairs.append((annotators[i], annotators[j]))
    return pairs

def assign_tasks(task_ids):
    annotators = ['vishnesh', 'ayush', 'vinayak', 'rachel']
    assignments = {annotator: [] for annotator in annotators}
    annotator_pairs = pairwise(annotators)
    random.seed(2)
    random.shuffle(task_ids)

    for i, task_id in enumerate(task_ids):
        first_annotator, second_annotator = annotator_pairs[i % len(annotator_pairs)]
        assignments[first_annotator].append(task_id)
        assignments[second_annotator].append(task_id)

    return assignments

task_ids = list(range(20))
assignments = assign_tasks(task_ids)
# sort the value
assignments = {k: sorted(v) for k, v in assignments.items()}

annotater_dict = {
    "vishnesh": [0, 1, 2, 3, 6, 7, 10, 13, 16, 17],
    "ayush": [2, 5, 7, 8, 9, 10, 11, 13, 14, 19],
    "vinayak": [0, 1, 4, 5, 6, 8, 9, 12, 15, 18],
    "rachel": [3, 4, 11, 12, 14, 15, 16, 17, 18, 19],
}

# print len of each annotator's batch
for annotator in annotater_dict:
    print(len(annotater_dict[annotator]))

# shuffle the annotator's batch
for annotator in annotater_dict:
    random.seed(2)
    random.shuffle(annotater_dict[annotator])


10
10
10
10


In [3]:
split_idx = 75

# Human 1 Writing,Human 1,Human 2 Writing,Human 2,GPT-3-zero-shot,GPT-3-few-shot,Muss,T5-3B,T5-11B
system_list = ["Davinci-003", "Turbo", "GPT-4", "Alpaca-7b", "Vicuna-7b"]
for annotater in annotater_dict:
    annotater_batch = []
    start_id = 0
    for id in annotater_dict[annotater]:
        # get the id th row of df
        row = df.iloc[id]
        random.seed(2)
        # random shuffle the system list
        random.shuffle(system_list)
        for system in system_list:
            current_dict = {}
            current_dict["id"] = start_id
            current_dict["original"] = row["Original Sentence"].strip()
            current_dict["original_spans"] = []

            simplified = row[system].strip()
            simplified = re.sub(r" 's", "'s", simplified)
            current_dict["simplified"] = simplified
            current_dict["simplified_spans"] = []

            if "||" in simplified:
                #  find index of all || in simplified
                indices = [m.start() for m in re.finditer('\|\|', simplified)]
                for i, indice in enumerate(indices):
                    current_dict["simplified_spans"].append([2, indice, indice+2, i])
            
            current_dict["system"] = f"new-wiki-{which_wiki}/{system}"
            annotater_batch += [current_dict]
            start_id += 1

    with open(f"../batches/new-wiki-{which_wiki}/{annotater}.json", "w") as f:
        json.dump(annotater_batch, f, indent=4)